In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class dual_svm:
    def __init__(self):
        self.epsilon = 1

    def check_constraint(self, y, alpha):
        length, _ = y.shape
        summation = 0
        
        # Generating alphas from 2 to m
        for i in range(1, length):
            summation += alpha[i] * y[i] * y[0]
            
        # Checking if the constraint is met or not    
        if -summation >= 0:
            return -summation
        else:
            return False
    
    def differentiation(self, x, y, alphas, i):
        length = alphas.shape[0]
        
        # Getting the summations ready
        first_summation, second_summation, third_summation, fourth_summation = 0, 0, 0, 0
        for j in range(1, length):
            first_summation += np.dot(alphas[j], (np.dot(y[j], y[0])))
            second_summation += np.dot(alphas[j], (np.dot(y[j], y[0])))
            third_summation += np.dot(alphas[j], np.dot(y[j], (1 + np.dot(x[j], x[0]))**2))
            if j != i:
                fourth_summation += np.dot(alphas[j], (np.dot(y[j], y[i])))
                
        # Third summation is a vector
        current_y_terms = np.dot(y[i], y[0])
        current_x_terms = (1 + np.dot(x[i], x[0]))**2
        
        # This is the 'a' part that would be returned
        final_term_svm = -current_y_terms + 1 - (first_summation * current_y_terms * 1 + (1 + np.dot(x[1], x[1]))**2) + y[0] \
        * ((first_summation * y[i] * current_x_terms) + (third_summation * current_y_terms)) \
        -((alphas[i] * (1 + np.dot(x[i], x[i]))**2) + (fourth_summation * (1 + np.dot(x[i], x[j]))**2))
        
#         print(final_term_svm)
        
        # This is the 'b' part to be returned
        final_term_barrier = ((1/alphas[i]) + ((y[i] * y[0]) / -first_summation))
        
        return final_term_svm, final_term_barrier
        
    
    def update_alphas(self, x, y ,alphas, time):
        
        alpha_lengths = alphas.shape[0]
        
        for i in range(1, alpha_lengths):
            # Finding differentiation of SVM part (a) and the log barrier part (b)
            a, b = self.differentiation(x, y, alphas, i)
            
            # Epsilon/time to decrease the power of barrier
            alphas[i] += a[0] + ((self.epsilon / time) * b)
            
        
    def barrier_svm(self, x, y):
        m, k = x.shape
        total_iterations = 100
        
        # Generating alphas satisfying the constraint
        while True:
            alpha_array = np.random.random_sample((m,))
            alpha1 = self.check_constraint(y, alpha_array)
            
            # If constraint is met, we change alpha_1 and break. Otherwise we continue
            if alpha1:
                alpha_array[0] = alpha1
                break
            
        # Got the initial feasible choice for alphas at t=0
        # Now computing the unconstrained maximizer using SVM's changes equation and barrier method.
        # self.barrier_equation(y, alpha_array)
        
#         print(alpha_array)
        
        for time in range(1, total_iterations):
#             print(time)
            # Using time to decrease the power of barrier (epsilon)
            
            self.update_alphas(x, y, alpha_array, time)
        
        print(alpha_array)
        

In [5]:
# Testing
d_svm = dual_svm()
d_svm.barrier_svm(np.array([[1, 1], [-1, 1], [-1, -1], [1, -1]]), np.array([[-1], [1], [-1], [1]]))

[ 6.73997722e-02 -1.52614823e+84 -7.69457161e+83  5.98956452e+86]
